In [ ]:
import torch, torchvision
from torchvision.datasets import MNIST
import torchvision.transforms as transforms
import torch.utils
from alexnet import AlexNet
import torch.nn as nn
from torch.optim import Adam

In [ ]:
W = 227  # minimum width for input to pass through alexnet
transform = transforms.Compose(
    [
        transforms.Resize(
            (W, W)
        ),  # the original mnist image size is 28*28, alexnet input size is 227*227
        transforms.ToTensor(),
        transforms.Normalize((0.5,), (0.5,)),
    ]
)

train = MNIST(root="./data", train=True, download=True, transform=transform)
test = MNIST(root="./data", train=False, download=True, transform=transform)

In [ ]:
batch_size = 64

train_loader = torch.utils.data.DataLoader(train, batch_size, shuffle=True)
test_loader = torch.utils.data.DataLoader(test, batch_size, shuffle=False)

model = AlexNet(10, 1)

In [ ]:
learning_rate = 0.01
num_epochs = 10
device = "cuda" if torch.cuda.is_available() else "cpu"

criterion = nn.CrossEntropyLoss()
optimizer = Adam(model.parameters(), lr=learning_rate)

In [ ]:
train_losses = []
test_losses = []
test_gen = enumerate(test_loader)

model = model.to(device)
for epoch in range(num_epochs):
    for idx, (train_imgs, train_labels) in enumerate(train_loader):
        train_imgs = train_imgs.to(device)
        train_labels = train_labels.to(device)

        # forward pass
        train_outputs = model(train_imgs)
        # print(labels.shape, outputs.shape, images.shape)
        train_loss = criterion(train_outputs, train_labels)
        train_losses.append(train_loss.item())
        # backward and optimize
        optimizer.zero_grad()
        train_loss.backward()
        optimizer.step()

        # get test loss
        try:
            _, (test_imgs, test_labels) = next(test_gen)
        except StopIteration:
            test_gen = enumerate(test_loader)
            _, (test_imgs, test_labels) = next(test_gen)

        test_imgs = test_imgs.to(device)
        test_labels = test_labels.to(device)

        test_outputs = model(test_imgs)
        test_loss = criterion(test_outputs, test_labels)
        test_losses.append(test_loss.item())

    print(f"epoch: {epoch} train_loss: {train_loss} test_loss: {test_loss}")

In [ ]:
import matplotlib.pyplot as plt

# plt.bar(range(len(train_losses)), train_losses, color='skyblue')

plt.bar(range(len(train_losses[400:1500])), train_losses[400:1500], color="skyblue")
plt.xlabel("Iteration")
plt.ylabel("Loss")
plt.title("Loss per Iteration")
plt.show()